In [ ]:
import re
import json
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

%matplotlib inline
# %matplotlib widget

## Map of Europe

In [ ]:
scale = 1 # 1 : 1,000,000
year = 2021
coord_ref = 3035 # coordinate reference system
                 # 3035: Lambert azimuthal equal area
                 # 3857: spherical Mercator projection
                 # 4326: world geodetic system 1984
europe_folder = f'geography/ref-nuts-{year}-{scale:02d}m'
N_levels = 4
map_types = 'BN', 'LB' # BN: boundary, LB: label, RG: region
europe = {map_type: {} for map_type in map_types}
for level in range(N_levels):
    gdf = {}
    for map_type in map_types:
        if map_type == 'LB':
            europe_file = f'{europe_folder}/NUTS_{map_type}_{year}_{coord_ref}_LEVL_{level}.json'
        else:
            europe_file = f'{europe_folder}/NUTS_{map_type}_{scale:02d}M_{year}_{coord_ref}_LEVL_{level}.json'
        gdf = gpd.read_file(europe_file)
        gdf.crs = CRS.from_user_input(coord_ref)
        europe[map_type][level] = gdf

## Coordinates of European capitals

In [ ]:
data = json.load(open('geography/country-capitals.json','r'))
blob = {'CNTR_NAME': [], 'CNTR_CODE': [], 'CITY_NAME': [], 'geometry': []}
for c in data:
    if c['ContinentName'] == 'Europe':
        blob['CNTR_NAME'].append(c['CountryName'])
        blob['CNTR_CODE'].append(c['CountryCode'])
        blob['CITY_NAME'].append(c['CapitalName'])
        blob['geometry'].append(Point(float(c['CapitalLongitude']), float(c['CapitalLatitude'])))
capitals = gpd.GeoDataFrame(data=blob)
capitals.crs = CRS.from_user_input(4326)
capitals = capitals.to_crs(epsg=coord_ref)
capitals_to_show = 'lisbon', 'madrid', 'paris', 'brussels', \
                   'amsterdam', 'rome', 'berlin', 'london', \
                   'dublin', 'stockholm', 'oslo', 'helsinki', \
                   'warsaw', 'bern', 'vienna', 'prague', 'sofia', \
                   'bucharest', 'budapest', 'riga', 'tallinn', \
                   'vilnius', 'reykjavik', 'athens', 'ankara', \
                   'nicosia'

## Bounding box

In [ ]:
bbox = {'WS': Point(-9.5, 30), 'EN': Point(75, 60)}
points = gpd.GeoDataFrame(data=bbox.values(),
                          index=pd.Index(data=bbox.keys(), name='name'),
                          columns=['geometry'])
points.loc['home'] = Point(9.400984, 44.272555)
points.crs = CRS.from_user_input(4326)
points = points.to_crs(epsg=coord_ref)

## Weather stations in Italy

In [ ]:
stations = pd.read_csv('ghcnd-stations.csv', header=None, index_col=0, \
                       names=['latitude','longitude', 'elevation', \
                              'name', 'prcp', 'prcp_attributes'])
# UK, GM, SP, IT, NO, SWE, FI
country_code = 'IT'
stations = stations.filter(regex='^'+country_code, axis='index')
stations['geometry'] = [Point(long, lat) for lat,long 
                        in stations.loc[:,['latitude','longitude']].itertuples(index=False)]
stations = gpd.GeoDataFrame(stations[['name','geometry']])
stations.crs = CRS.from_user_input(4326)
stations = stations.to_crs(epsg=coord_ref)
print(f'There are {len(stations)} weather stations in {country_code}.')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(12,8))
light_gray = .75 + np.zeros(3)
dark_gray = .3 + np.zeros(3)
europe['BN'][0].plot(ax=ax, lw=1, color=dark_gray)
europe['BN'][1].plot(ax=ax, lw=0.75, color=light_gray)
europe['BN'][2].plot(ax=ax, lw=0.75, color=light_gray)
europe['BN'][3].plot(ax=ax, lw=0.75, color=light_gray)
for point,ID in zip(europe['LB'][0].geometry, europe['LB'][0].NUTS_ID):
    ax.text(point.x, point.y, ID,
            horizontalalignment='center',
            verticalalignment='center',
            fontsize=14, color=[.7,.2,.2])
for point,name in zip(capitals.geometry, capitals.CITY_NAME):
    ax.plot(point.x, point.y, 's', markersize=6, color=light_gray)
    if name.lower() in capitals_to_show:
        ax.text(point.x, point.y, name.capitalize(),
                horizontalalignment='left',
                verticalalignment='baseline',
                fontsize=10, color=[0,.5,1])
for point in stations.geometry:
    ax.plot(point.x, point.y, '.', markersize=5, color=[0,1,.5])
ax.set_xlim([points.loc['WS','geometry'].x, points.loc['EN','geometry'].x])
ax.set_ylim([points.loc['WS','geometry'].y, points.loc['EN','geometry'].y])
ax.axis('off')
fig.tight_layout()